In [1]:
# init
%config IPCompleter.greedy = True  
%matplotlib inline

import pandas as pd
import numpy as np

# Importeer dataset, houd rekening met encoding(!)
csvdata = pd.read_csv('Data\pc4dipv2-csvdata.csv', sep=';',encoding='ISO-8859-1').set_index(['nr'])
pcdata = pd.read_csv('Data\PC4-DB.csv', sep=';',encoding='UTF-8')

# Importeer postcodedatabase

In [2]:
# exploreer data (Zijn los getest - samengevoegd voor leesbaarheid)
csvdata.head()
csvdata.info()

pcdata.head()
pcdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398632 entries, 1 to 398632
Data columns (total 13 columns):
PC4_LEERL             398632 non-null int64
KWALIFICATIENIVEAU    398632 non-null int64
KWALIFICATIE CODE     398632 non-null int64
KWALIFICATIE NAAM     398632 non-null object
MBO SECTOR            398632 non-null object
DOMEIN                398632 non-null object
SECTORUNIT SBB        398632 non-null object
GESLACHT              398632 non-null object
2013   AANT           398632 non-null int64
2014   AANT           398632 non-null int64
2015   AANT           398632 non-null int64
2016   AANT           398632 non-null int64
2017   AANT           398632 non-null int64
dtypes: int64(8), object(5)
memory usage: 42.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4307 entries, 0 to 4306
Data columns (total 9 columns):
PC4                 4307 non-null int64
Woonplaatscode      4307 non-null int64
Woonplaats          4307 non-null object
Gemeentecode        4307 non-null 

In [3]:
# Zijn er records met lege waarden? (Beide zijn los getest - samengevoegd voor leesbaarheid)
csvdata.isnull().any().any()
pcdata.isnull().any().any()

False

In [4]:
# Hoeveel kwalificatiecodes zijn er?
kc = csvdata['KWALIFICATIE CODE'].unique()
kc.size

1222

In [5]:
# En hoeveel beschrijvingen zijn er?
kn = csvdata['KWALIFICATIE NAAM'].unique()
kn.size

1080

In [6]:
# Welke kwalificaties hebben meerdere kwalificatie codes?
dup = csvdata.groupby(['KWALIFICATIE NAAM', 'KWALIFICATIE CODE'], as_index=False).sum()[['KWALIFICATIE NAAM','KWALIFICATIE CODE']]
dup[dup.duplicated(['KWALIFICATIE NAAM'], keep=False)]

,KWALIFICATIE NAAM,KWALIFICATIE CODE
2,AV-productie,90060
3,AV-productie,90070
4,AV-productie (Allround medewerker AV-productie),90061
5,AV-productie (Allround medewerker AV-productie),90063
6,AV-productie (Beeldtechnicus),90071
7,AV-productie (Beeldtechnicus),90081
8,AV-productie (Cameraman),90073
9,AV-productie (Cameraman),90082
10,AV-productie (Fotograaf),90075
11,AV-productie (Fotograaf),90076


In [7]:
# Controleer op unieke postcodes in dataset
csvuniquepc = csvdata['PC4_LEERL'].unique()
csvuniquepc

array([  10,   20,   30, ..., 9997, 9998, 9999], dtype=int64)

In [8]:
# Hoeveel unieke postcodes zijn er?
pcuniquepc = pcdata['PC4'].unique()
pcuniquepc

array([9446, 9465, 9467, ..., 8043, 8044, 8045], dtype=int64)

In [13]:
# Welke postcodes zijn ongeldig?
pcdiff = set(csvuniquepc) - set(pcuniquepc)
pcdiff

{10, 20, 30, 40, 1800, 3300, 6130, 7680}

In [10]:
# Hoeveel postcodes zijn niet te matchen met elkaar?
df = csvdata[csvdata['PC4_LEERL'].isin(pcdiff)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1390 entries, 1 to 324342
Data columns (total 13 columns):
PC4_LEERL             1390 non-null int64
KWALIFICATIENIVEAU    1390 non-null int64
KWALIFICATIE CODE     1390 non-null int64
KWALIFICATIE NAAM     1390 non-null object
MBO SECTOR            1390 non-null object
DOMEIN                1390 non-null object
SECTORUNIT SBB        1390 non-null object
GESLACHT              1390 non-null object
2013   AANT           1390 non-null int64
2014   AANT           1390 non-null int64
2015   AANT           1390 non-null int64
2016   AANT           1390 non-null int64
2017   AANT           1390 non-null int64
dtypes: int64(8), object(5)
memory usage: 152.0+ KB


In [11]:
# Verwijder dataregels uit dataset die geen overeenkomende postcode hebben
newcsvdata = csvdata[~csvdata['PC4_LEERL'].isin(pcdiff)]
newcsvdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397242 entries, 1387 to 398632
Data columns (total 13 columns):
PC4_LEERL             397242 non-null int64
KWALIFICATIENIVEAU    397242 non-null int64
KWALIFICATIE CODE     397242 non-null int64
KWALIFICATIE NAAM     397242 non-null object
MBO SECTOR            397242 non-null object
DOMEIN                397242 non-null object
SECTORUNIT SBB        397242 non-null object
GESLACHT              397242 non-null object
2013   AANT           397242 non-null int64
2014   AANT           397242 non-null int64
2015   AANT           397242 non-null int64
2016   AANT           397242 non-null int64
2017   AANT           397242 non-null int64
dtypes: int64(8), object(5)
memory usage: 42.4+ MB
